In [1]:
import pandas as pd
from striprtf.striprtf import rtf_to_text
import re
import os

In [2]:
rtf_file_path = r"C:\Users\Windows10\Documents\GitHub\Kenya\Worlddev\RichTextFiles\mcd_DHF_47.rtf"  

In [3]:
with open(rtf_file_path, "r", encoding="utf-8") as file:
    rtf_content = file.read()

# Convert the RTF content to plain text
plain_text = rtf_to_text(rtf_content)

# Split the content into lines
lines = plain_text.split('\n')



In [27]:
# Extract the year and handle it dynamically
header_line = lines[0]  # Assuming the first line contains the header with year
year_match = re.search(r'Thailand\s+(\d{4})\s+\((\d{4})\)', header_line) 
#Using Regular Expression to extract the name to filter
'''
1) Literal string 'Thailand' (from file name)
2) \s+ represents a whitespace character, and + selects one or more of these characters.
3) \d means select from 0-9 but with (\d{4}) means select exactly 4 digits
4) \s+ another whitespace
5) \( get the parenthesis
6) (\d{4}) get another exactly 4 digits group
7 \) get the last parenthesis 
'''


if year_match:
    buddhist_year = year_match.group(1)
    gregorian_year = year_match.group(2)
    print(f"Buddhist year(ปีพ.ศ.) {buddhist_year}, Gregorian year(ปีค.ศ.) {gregorian_year}")

In [5]:
lines = lines[1:]

# Initialize lists to hold data for each row
reporting_areas = []
total_cases = []
total_deaths = []
monthly_cases = {month: [] for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']}
monthly_deaths = {month: [] for month in ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'June', 'July', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']}

In [6]:
for line in lines:
    columns = line.split('\t')  # Split based on tabs or spaces

    if len(columns) >= 3:  # Ensure the row has at least reporting area, total cases, and total deaths
        # First column is the Reporting Area
        if not columns[0].strip():
            # Shift columns to the left by 1 to align Dec_case and Dec_death
            reporting_areas.append(columns[1])
            columns = columns[1:]  # Remove the first empty column
        else:
            reporting_areas.append(columns[0])

        # Handle total cases and deaths
        total_cases.append(columns[1] if len(columns) > 1 else None)
        total_deaths.append(columns[2] if len(columns) > 2 else None)
        
        # Handle monthly cases and deaths
        for i, month in enumerate(monthly_cases.keys()):
            # Check if the index is within range before accessing
            case_index = 3 + i * 2
            death_index = 4 + i * 2
            
            if len(columns) > case_index:
                monthly_cases[month].append(columns[case_index])  # Cases
            else:
                monthly_cases[month].append(None)  # If data is missing, append None

            if len(columns) > death_index:
                monthly_deaths[month].append(columns[death_index])  # Deaths
            else:
                monthly_deaths[month].append(None) 

In [7]:
data = {
    'Reporting areas': reporting_areas,
    'Total_case': total_cases,
    'Total_death': total_deaths,
}

# Add monthly cases and deaths to the data
for month in monthly_cases:
    data[f'{month}_case'] = monthly_cases[month]
    data[f'{month}_death'] = monthly_deaths[month]

In [8]:
df = pd.DataFrame(data)

In [9]:
if df.shape[1] > 27:
    extra_column = df.columns[-1]
    df['Dec_death'] = df[extra_column]
    df = df.drop(extra_column, axis=1)

In [10]:
df

,Reporting areas,Total_case,Total_death,Jan_case,Jan_death,Feb_case,Feb_death,Mar_case,Mar_death,Apr_case,...,Aug_case,Aug_death,Sep_case,Sep_death,Oct_case,Oct_death,Nov_case,Nov_death,Dec_case,Dec_death
0,Reporting areas,Total,Jan,Feb,Mar,Apr,May,June,July,Aug,...,None,None,None,None,None,None,None,None,None,None
1,cases,death,case,deat,case,death,case,deat,case,deat,...,deat,case,death,case,death,case,death,case,death,None
2,Total,27111,12,1045,0,892,1,1239,1,1277,...,3948,1,3117,1,2375,1,2338,0,1238,0
3,North Region,4042,4,97,0,96,1,130,1,193,...,694,0,398,1,191,0,150,0,83,0
4,Zone:1,1432,1,4,0,8,0,27,0,44,...,251,0,132,1,50,0,37,0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
104,Yala,190,0,10,0,14,0,4,0,1,...,31,0,21,0,13,0,21,0,8,0
105,Zone:19,652,0,63,0,26,0,23,0,20,...,66,0,55,0,34,0,87,0,55,0
106,Satun,140,0,4,0,2,0,1,0,5,...,14,0,12,0,8,0,21,0,12,0
107,Songkhla,512,0,59,0,24,0,22,0,15,...,52,0,43,0,26,0,66,0,43,0


In [11]:
df = df.drop([0, 1, 108]).reset_index(drop=True)

In [12]:
df

,Reporting areas,Total_case,Total_death,Jan_case,Jan_death,Feb_case,Feb_death,Mar_case,Mar_death,Apr_case,...,Aug_case,Aug_death,Sep_case,Sep_death,Oct_case,Oct_death,Nov_case,Nov_death,Dec_case,Dec_death
0,Total,27111,12,1045,0,892,1,1239,1,1277,...,3948,1,3117,1,2375,1,2338,0,1238,0
1,North Region,4042,4,97,0,96,1,130,1,193,...,694,0,398,1,191,0,150,0,83,0
2,Zone:1,1432,1,4,0,8,0,27,0,44,...,251,0,132,1,50,0,37,0,7,0
3,Chiang Mai,504,1,0,0,3,0,4,0,6,...,129,0,92,1,29,0,15,0,3,0
4,Chiang Rai,79,0,0,0,0,0,1,0,2,...,11,0,3,0,3,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,Pattani,147,0,14,0,14,0,2,0,1,...,17,0,16,0,18,0,19,0,7,0
102,Yala,190,0,10,0,14,0,4,0,1,...,31,0,21,0,13,0,21,0,8,0
103,Zone:19,652,0,63,0,26,0,23,0,20,...,66,0,55,0,34,0,87,0,55,0
104,Satun,140,0,4,0,2,0,1,0,5,...,14,0,12,0,8,0,21,0,12,0
